In [1]:
import os
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np
import math
class SpeedDataset(Dataset):
    def __init__(self, directory, sequence_length=5):
        self.data = []
        self.sequence_length = sequence_length
        self.max_speed = 0  # Variable to store the maximum speed found
        self.preprocess_data(directory)
        # Calculate number of classes based on the range from 30 to 105
        self.num_classes = (105 - 30) // 10 + 1  # From class 30-39, ..., 100-105

    def preprocess_data(self, directory):
        for filename in os.listdir(directory):
            if filename.endswith('.txt'):
                speed = float(filename.split('_')[-1].replace('.txt', ''))
                # Update maximum speed if current speed is greater
                if speed > self.max_speed:
                    self.max_speed = speed
                # Convert speed to class, starting range from 30
                if speed < 30:
                    speed_class = 0  # Handle speeds below 30 as class 0
                else:
                    speed_class = (int(speed) - 30) // 10
                filepath = os.path.join(directory, filename)
                with open(filepath, 'r') as file:
                    track_data = {}
                    for line in file:
                        frame, track_id, x1, y1, x2, y2 = map(float, line.strip().split(',')[:6])
                        if track_id not in track_data:
                            track_data[track_id] = []
                        track_data[track_id].append([x1,y1,x2,y2,(x1+x2)/2, (y1+y2)/2])

                    for track_id, frames in track_data.items():
                        if len(frames) >= self.sequence_length:
                            features = []
                            for i in range(1, len(frames)):
                                current_frame = frames[i]
                                previous_frame = frames[i-1]
                                data_d = []
                                distance = 0
                                for i in range(4):
                                    data_d.append(abs(current_frame[i] - previous_frame[i]))
                                for j in range(4,6):
                                    distance = ((current_frame[j] - previous_frame[j])**2) + distance
                                data_d.append(math.sqrt(distance))
                                features.append(data_d)
                                
                            for i in range(len(features) - self.sequence_length + 1):
                                sequence = features[i:i + self.sequence_length]
                                self.data.append((sequence, speed_class))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        inputs, output = self.data[idx]
        return torch.tensor(inputs, dtype=torch.float32), torch.tensor(output, dtype=torch.long)


In [15]:
import torch
import torch.nn as nn

class SpeedPredictor(nn.Module):
    def __init__(self, sequence_length, feature_size, embedding_dim, hidden_dim, output_size):
        super(SpeedPredictor, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=feature_size, out_channels=embedding_dim, kernel_size=1)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.fc2 = nn.Linear(hidden_dim, output_size)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = x.permute(0, 2, 1)
        x = self.conv1(x)
        x = x.permute(0, 2, 1)
        x, _ = self.lstm(x)
        x = x[:, -1, :]
        return self.relu(self.fc2(x))


In [3]:
import torch

# Check for CUDA
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cuda


In [4]:
def train(model, train_loader, test_loader, criterion, optimizer, epochs):
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        correct_train = 0
        total_train = 0

        for inputs, targets in train_loader:
            inputs, targets = inputs.to(device), targets.to(device)  # Ensure inputs and targets are on the same device as model
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

            _, predicted = torch.max(outputs.data, 1)
            correct_train += (predicted == targets).sum().item()
            total_train += targets.size(0)

        train_accuracy = 100 * correct_train / total_train

        if epoch % 30 == 0 or epoch == epochs - 1:
            model.eval()
            correct_test = 0
            total_test = 0
            all_preds = []
            all_targets = []
            with torch.no_grad():
                for inputs, targets in test_loader:
                    inputs, targets = inputs.to(device), targets.to(device)  # Ensure inputs and targets are on the same device as model
                    outputs = model(inputs)
                    _, predicted = torch.max(outputs.data, 1)
                    correct_test += (predicted == targets).sum().item()
                    total_test += targets.size(0)
                    all_preds.extend(predicted.cpu().numpy())
                    all_targets.extend(targets.cpu().numpy())
            test_accuracy = 100 * correct_test / total_test
            print(f'Epoch {epoch+1}: Train Loss: {total_loss / len(train_loader)}, '
                  f'Train Accuracy: {train_accuracy:.2f}%, Test Accuracy: {test_accuracy:.2f}%')



In [12]:
train_dataset = SpeedDataset('outputs', sequence_length=5)
test_dataset = SpeedDataset('test', sequence_length=5)
max_classes = max(train_dataset.num_classes, test_dataset.num_classes)
model = SpeedPredictor(sequence_length=5, feature_size=5, embedding_dim=64, hidden_dim=64, output_size = max_classes)
model.to(device)  # Move model to the appropriate device
criterion = nn.CrossEntropyLoss().to(device)  # Move criterion to the appropriate device
optimizer = torch.optim.SGD(model.parameters(), lr=0.05)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=10, shuffle=False)
train(model, train_loader, test_loader, criterion, optimizer, epochs=300)


Epoch 1: Train Loss: 2.026758436106334, Train Accuracy: 18.58%, Test Accuracy: 14.63%
Epoch 11: Train Loss: 1.9194468127356634, Train Accuracy: 22.71%, Test Accuracy: 17.78%
Epoch 21: Train Loss: 1.8880309782065379, Train Accuracy: 24.16%, Test Accuracy: 17.59%
Epoch 31: Train Loss: 1.8702795733252928, Train Accuracy: 24.67%, Test Accuracy: 16.25%
Epoch 41: Train Loss: 1.8541667082621107, Train Accuracy: 26.17%, Test Accuracy: 18.31%
Epoch 51: Train Loss: 1.8163501089311715, Train Accuracy: 28.30%, Test Accuracy: 17.97%
Epoch 61: Train Loss: 1.7810241636941773, Train Accuracy: 30.12%, Test Accuracy: 18.45%
Epoch 71: Train Loss: 1.744457274617275, Train Accuracy: 31.80%, Test Accuracy: 21.75%
Epoch 81: Train Loss: 1.6895972137097959, Train Accuracy: 34.90%, Test Accuracy: 19.69%
Epoch 91: Train Loss: 1.6372859080632527, Train Accuracy: 37.34%, Test Accuracy: 18.79%
Epoch 101: Train Loss: 1.5904273259477075, Train Accuracy: 39.72%, Test Accuracy: 18.55%
Epoch 111: Train Loss: 1.521412711

In [13]:
def print_confusion_matrix_and_report(all_targets, all_preds):
    print(confusion_matrix(all_targets, all_preds))
    print(classification_report(all_targets, all_preds, target_names=[f'Class {30 + i * 10}-{39 + i * 10}' for i in range(max(all_targets) + 1)]))


In [14]:
def evaluate(model, test_loader):
    model.eval()
    correct_test = 0
    total_test = 0
    all_preds = []
    all_targets = []
    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)  # Ensure inputs and targets are on the same device as model
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            correct_test += (predicted == targets).sum().item()
            total_test += targets.size(0)
            all_preds.extend(predicted.cpu().numpy())
            all_targets.extend(targets.cpu().numpy())

    test_accuracy = 100 * correct_test / total_test
    print("Confusion Matrix and Classification Report:")
    print_confusion_matrix_and_report(all_targets, all_preds)
    return test_accuracy
evaluate(model, test_loader)  

Confusion Matrix and Classification Report:
[[106  53 100  44  40  18  23  11]
 [107  58  50  42  23   6   8   9]
 [ 61  54  58  76  25   5  10   2]
 [ 37  32  50  72  21  19  22   7]
 [ 18  19  39  40  29  22  50   5]
 [ 28  15  26  34  24  12  55  14]
 [ 26  12  27  34  27   4  46   3]
 [  6  19  37  26  47  13  57  29]]
               precision    recall  f1-score   support

  Class 30-39       0.27      0.27      0.27       395
  Class 40-49       0.22      0.19      0.21       303
  Class 50-59       0.15      0.20      0.17       291
  Class 60-69       0.20      0.28      0.23       260
  Class 70-79       0.12      0.13      0.13       222
  Class 80-89       0.12      0.06      0.08       208
  Class 90-99       0.17      0.26      0.20       179
Class 100-109       0.36      0.12      0.18       234

     accuracy                           0.20      2092
    macro avg       0.20      0.19      0.18      2092
 weighted avg       0.21      0.20      0.19      2092



19.59847036328872